In [ ]:
import numpy as np;import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,OneHotEncoder,LabelEncoder
from sklearn.model_selection import cross_val_predict,GridSearchCV
from sklearn.metrics import * 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,VotingClassifier
import xgboost as xgb

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Load Data
import pandas as pd
train=pd.read_csv('/content/gdrive/MyDrive/Laduma/Train_Laduma.csv',na_values='NULL')
test=pd.read_csv('/content/gdrive/MyDrive/Laduma/Test_Laduma.csv',na_values='NULL')
train_game_statistics=pd.read_csv('/content/gdrive/MyDrive/Laduma/train_game_statistics.csv',na_values='NULL')
test_game_statistics=pd.read_csv('/content/gdrive/MyDrive/Laduma/test_game_statistics.csv',na_values='NULL')
sub_data=pd.read_csv('/content/gdrive/MyDrive/Laduma/SampleSubmission_Laduma.csv',na_values='NULL')
train.shape,test.shape

((412, 7), (234, 6))

In [ ]:
train_game_statistics.duplicated().sum()

3743

In [ ]:
train_game_statistics.drop_duplicates(keep='first',inplace=True)

In [ ]:
train_game_statistics.drop(['next_player', 'next_x', 'next_y', 'event_id', 'next_team','next_event_id', 'xt_value'], axis=1, inplace=True)

In [ ]:
train_game_statistics.drop(['next_action'], axis=1, inplace=True)

In [ ]:
temp_data=list(zip(train.Game_ID,train['Home Team']))
temp_data

[('ID_KAG4KAE9', 'Antennae'),
 ('ID_T7H1XN5Z', 'Andromeda'),
 ('ID_O1P9FGN4', 'Andromeda'),
 ('ID_TR5M4KQV', 'Andromeda'),
 ('ID_EOIECRE2', 'Andromeda'),
 ('ID_8CRFDQZA', 'Andromeda'),
 ('ID_CV7BOXYD', 'Andromeda'),
 ('ID_X5UEAAIY', 'Andromeda'),
 ('ID_Z8TGBH41', 'Andromeda'),
 ('ID_4CYGD80A', 'Andromeda'),
 ('ID_XQMJ05QY', 'Andromeda'),
 ('ID_OW9Z59TM', 'Andromeda'),
 ('ID_QEHB6SP5', 'Butterfly'),
 ('ID_UL9DKAH4', 'Cartwheel'),
 ('ID_UFDKE17M', 'Sculptor'),
 ('ID_GU6BX7OY', 'Cigar'),
 ('ID_S6RUK5NW', 'Comet'),
 ('ID_VUN0XPN1', 'Cosmos Redshift 7'),
 ('ID_XCHM541T', 'Eye of Sauron'),
 ('ID_9QKWORXG', 'Medusa Merger'),
 ('ID_TLXXQLYC', 'Milky Way'),
 ('ID_HPYKEW7R', 'Antennae'),
 ('ID_IZLDEI78', 'Andromeda'),
 ('ID_UC7PMJ25', 'Andromeda'),
 ('ID_5HWA04IU', 'Andromeda'),
 ('ID_20BPYE4G', 'Andromeda'),
 ('ID_CHH8Y1BD', 'Andromeda'),
 ('ID_T1Q3B3GG', 'Andromeda'),
 ('ID_DEBRT4C8', 'Andromeda'),
 ('ID_9DPBXX3W', 'Andromeda'),
 ('ID_DF19AH9G', 'Andromeda'),
 ('ID_K81P666R', 'Andromeda'),
 ('

In [ ]:
home_data=[];
for x,y in temp_data:
    b=train_game_statistics[(train_game_statistics.Game_ID==x) & (train_game_statistics.Team==y)].values
    home_data.append(b)
home_data[0]

array([['ID_KAG4KAE9', 'Player_0BOHJ6V2', 6.0, ..., 1.0, 0.28, 0.62],
       ['ID_KAG4KAE9', 'Player_0BOHJ6V2', 7.0, ..., 0.0, 0.28, 0.62],
       ['ID_KAG4KAE9', 'Player_0BOHJ6V2', 8.0, ..., 0.0, 0.28, 0.62],
       ...,
       ['ID_KAG4KAE9', 'Player_3IA2L056', 3722.0, ..., 0.0, 94.05, 94.38],
       ['ID_KAG4KAE9', 'Player_0A2NADNO', 3726.0, ..., 0.0, 94.1, 94.43],
       ['ID_KAG4KAE9', 'Player_0A2NADNO', 3727.0, ..., 0.0, 94.1, 94.43]],
      dtype=object)

In [ ]:
# Away Team
temp_data=list(zip(train.Game_ID,train['Away Team']))
temp_data

[('ID_KAG4KAE9', 'Andromeda'),
 ('ID_T7H1XN5Z', 'Antennae'),
 ('ID_O1P9FGN4', 'Butterfly'),
 ('ID_TR5M4KQV', 'Cigar'),
 ('ID_EOIECRE2', 'Circinus'),
 ('ID_8CRFDQZA', 'Cartwheel'),
 ('ID_CV7BOXYD', 'Coma Pinwheel'),
 ('ID_X5UEAAIY', 'Comet'),
 ('ID_Z8TGBH41', 'Cosmos Redshift 7'),
 ('ID_4CYGD80A', 'Fireworks'),
 ('ID_XQMJ05QY', 'Medusa Merger'),
 ('ID_OW9Z59TM', 'Milky Way'),
 ('ID_QEHB6SP5', 'Andromeda'),
 ('ID_UL9DKAH4', 'Andromeda'),
 ('ID_UFDKE17M', 'Andromeda'),
 ('ID_GU6BX7OY', 'Andromeda'),
 ('ID_S6RUK5NW', 'Andromeda'),
 ('ID_VUN0XPN1', 'Andromeda'),
 ('ID_XCHM541T', 'Andromeda'),
 ('ID_9QKWORXG', 'Andromeda'),
 ('ID_TLXXQLYC', 'Andromeda'),
 ('ID_HPYKEW7R', 'Andromeda'),
 ('ID_IZLDEI78', 'Antennae'),
 ('ID_UC7PMJ25', 'Backward'),
 ('ID_5HWA04IU', 'Sculptor'),
 ('ID_20BPYE4G', 'Cartwheel'),
 ('ID_CHH8Y1BD', 'Sunflower'),
 ('ID_T1Q3B3GG', 'Comet'),
 ('ID_DEBRT4C8', 'Cosmos Redshift 7'),
 ('ID_9DPBXX3W', 'Medusa Merger'),
 ('ID_DF19AH9G', 'Milky Way'),
 ('ID_K81P666R', 'Triangulum

In [ ]:
away_data=[];
for x,y in temp_data:
    b=train_game_statistics[(train_game_statistics.Game_ID==x) & (train_game_statistics.Team==y)].values
    away_data.append(b)
away_data[0]

array([['ID_KAG4KAE9', 'Player_MV0TGOYE', 1.0, ..., 0.0, 0.18, 0.52],
       ['ID_KAG4KAE9', 'Player_MV0TGOYE', 2.0, ..., 1.0, 0.18, 0.52],
       ['ID_KAG4KAE9', 'Player_P1I1LJ4T', 3.0, ..., 0.0, 0.2, 0.53],
       ...,
       ['ID_KAG4KAE9', 'Player_D9UE7ID1', 3724.0, ..., 1.0, 94.08, 94.42],
       ['ID_KAG4KAE9', 'Player_CFXW866O', 3725.0, ..., 0.0, 94.1, 94.43],
       ['ID_KAG4KAE9', 'Player_CFXW866O', 3728.0, ..., 0.0, 94.1, 94.43]],
      dtype=object)

In [ ]:
# Convert Lists to DataFrames
# Home Data
b=[pd.DataFrame(home_data[x],columns=train_game_statistics.columns) for x in range(len(home_data))]
home_data1=pd.concat(b)
print(home_data1.shape)
# Away data
b=[pd.DataFrame(away_data[x],columns=train_game_statistics.columns) for x in range(len(away_data))]
away_data1=pd.concat(b)
print(away_data1.shape)

(796633, 20)
(771200, 20)


In [ ]:
game_home=home_data1.groupby(['Game_ID','Player_ID']).sum().groupby(['Game_ID']).sum()
game_home

,id,X,Y,Season,Shots,SoT,Goals_scored,Goals_conceded,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes
Game_ID,,,,,,,,,,,,,
ID_00EFNL7L,3321595.0,101287.8,62556.1,1826,19.0,7.0,0.0,1.0,327.0,83.0,410.0,93684.54,94291.59
ID_00J124MI,2939691.0,88784.6,53127.2,1615,18.0,6.0,0.0,0.0,259.0,71.0,330.0,67403.75,67941.73
ID_04T9W310,3700349.0,88821.0,71387.8,3886,6.0,4.0,2.0,2.0,394.0,87.0,481.0,88518.80,89169.40
ID_06QXUEYD,4277256.0,113904.2,67590.5,4152,11.0,2.0,2.0,1.0,448.0,82.0,530.0,104091.11,104784.62
ID_0A2WEVY3,3797786.0,87106.4,59946.5,3662,4.0,2.0,0.0,1.0,376.0,99.0,475.0,102341.48,102952.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_ZWD4FQ05,3572688.0,104685.0,56358.4,1956,7.0,5.0,2.0,2.0,324.0,94.0,418.0,89890.41,90545.37
ID_ZWJJP6T6,3052205.0,88078.9,54625.1,3430,10.0,3.0,2.0,1.0,279.0,77.0,356.0,75182.42,75756.45
ID_ZWPPN4W1,4404438.0,114540.9,68772.6,2078,11.0,2.0,0.0,1.0,352.0,93.0,445.0,104357.39,105050.28


In [ ]:
game_home.drop(['id','X','Y','Season'], axis=1, inplace=True)

In [ ]:
game_home['Shot_Pct']=((game_home['SoT']+1)/(game_home['Shots']+1))
game_home['Pass_Pct']=((game_home['Accurate passes']+1)/(game_home['Passes']+1))

In [ ]:
game_away=away_data1.groupby(['Game_ID','Player_ID']).sum().groupby(['Game_ID']).sum()
game_away

,id,X,Y,Season,Shots,SoT,Goals_scored,Goals_conceded,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes
Game_ID,,,,,,,,,,,,,
ID_00EFNL7L,2657258.0,80624.4,53096.2,1630,6.0,0.0,1.0,0.0,263.0,86.0,349.0,75153.18,75697.46
ID_00J124MI,4525325.0,123879.0,71149.4,2247,12.0,3.0,0.0,0.0,501.0,76.0,577.0,104647.57,105395.54
ID_04T9W310,4415985.0,120265.1,73400.3,4168,9.0,2.0,2.0,2.0,407.0,95.0,502.0,105823.38,106521.65
ID_06QXUEYD,3076214.0,93232.3,66253.9,3514,8.0,3.0,1.0,2.0,352.0,72.0,424.0,73707.88,74295.43
ID_0A2WEVY3,3207252.0,104798.2,70937.9,3820,12.0,3.0,1.0,0.0,390.0,99.0,489.0,85326.07,85963.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_ZWD4FQ05,3114015.0,86678.6,59285.2,1699,10.0,4.0,2.0,2.0,243.0,97.0,340.0,78474.32,79043.73
ID_ZWJJP6T6,3949008.0,104160.0,66482.6,4050,11.0,3.0,1.0,2.0,414.0,81.0,495.0,97547.29,98224.11
ID_ZWPPN4W1,4248447.0,102570.4,71206.0,2080,6.0,2.0,1.0,0.0,364.0,99.0,463.0,100736.08,101430.25


In [ ]:
game_away.drop(['id','X','Y','Season'], axis=1, inplace=True)

In [ ]:
game_away['Shot_Pct']=((game_away['SoT']+1)/(game_away['Shots']+1))
game_away['Pass_Pct']=((game_away['Accurate passes']+1)/(game_away['Passes']+1))

In [ ]:
#game_combined=game_home

In [ ]:
#game_combined['Shots_ratio']=(game_home['Shots']+1)/(game_away['Shots']+1)

In [ ]:
#game_combined['SoT_ratio']=(game_home['SoT']+1)/(game_away['SoT']+1)
#game_combined['acc_pass_ratio']=(game_home['Accurate passes'])/(game_away['Accurate passes'])
#game_combined['inacc_pass_ratio']=(game_home['Inaccurate passes'])/(game_away['Inaccurate passes'])
#game_combined['pass_ratio']=(game_home['Passes'])/(game_away['Passes'])
#game_combined['Shotpct_ratio']=(game_home['Shot_Pct'])/(game_away['Shot_Pct'])
#game_combined['passpct_ratio']=(game_home['Pass_Pct'])/(game_away['Pass_Pct'])

Creating new columns from Goals Scored and Goals conceded

In [ ]:
home=pd.merge(train,game_home,left_on='Game_ID',right_index=True,how='inner')

In [ ]:
away=pd.merge(train,game_away,left_on='Game_ID',right_index=True,how='inner')

In [ ]:
home1=home[['Home Team', 'Goals_scored', 'Goals_conceded']].copy()

In [ ]:
away1=away[['Away Team', 'Goals_scored', 'Goals_conceded']].copy()

In [ ]:
home2=home1.groupby(['Home Team']).sum()

In [ ]:
away2=away1.groupby(['Away Team']).sum()

In [ ]:
home2['Goals_scored']=home2['Goals_scored']+away2['Goals_scored']

In [ ]:
home2['Goals_conceded']=home2['Goals_conceded']+away2['Goals_conceded']

In [ ]:
home2['Goal_diff']=home2['Goals_scored']-home2['Goals_conceded']

In [ ]:
home2['Goals_ratio']=home2['Goals_scored']/home2['Goals_conceded']

In [ ]:
home2=home2.reset_index()

In [ ]:
goals=home2

In [ ]:
goals.rename(columns={'Home Team':'Team'}, inplace=True)

In [ ]:
game_home.drop(['Goals_scored','Goals_conceded'], axis=1, inplace=True)

In [ ]:
game_away.drop(['Goals_scored','Goals_conceded'], axis=1, inplace=True)

In [ ]:
game_combined=game_home

In [ ]:
game_combined['Shots_ratio']=(game_home['Shots']+1)/(game_away['Shots']+1)

In [ ]:
game_combined['SoT_ratio']=(game_home['SoT']+1)/(game_away['SoT']+1)
game_combined['acc_pass_ratio']=(game_home['Accurate passes'])/(game_away['Accurate passes'])
game_combined['inacc_pass_ratio']=(game_home['Inaccurate passes'])/(game_away['Inaccurate passes'])
game_combined['pass_ratio']=(game_home['Passes'])/(game_away['Passes'])

In [ ]:
train_data1=pd.merge(train,game_combined,left_on='Game_ID',right_index=True,how='inner')

In [ ]:
goals1=goals

In [ ]:
goals1.rename(columns={'Goals_ratio':'Goals_ratio_away'}, inplace=True)

In [ ]:
goals1['Goals_ratio_home']=goals1['Goals_ratio_away']

In [ ]:
goals2=goals1[['Team', 'Goals_ratio_away']].copy()

In [ ]:
goals1.drop(['Goals_ratio_away'], axis=1, inplace=True)

In [ ]:
goals1.rename(columns={'Team':'Home Team'}, inplace=True)
goals2.rename(columns={'Team':'Away Team'}, inplace=True)

In [ ]:
goals1.head()

,Home Team,Goals_scored,Goals_conceded,Goal_diff,Goals_ratio_home
0,Andromeda,47.0,48.0,-1.0,0.979167
1,Antennae,53.0,50.0,3.0,1.060000
2,Backward,41.0,51.0,-10.0,0.803922
3,Butterfly,28.0,21.0,7.0,1.333333
4,Cartwheel,72.0,72.0,0.0,1.000000


Merging

In [ ]:
train_data=pd.merge(train_data1,goals1, how='inner', left_on='Home Team', right_on='Home Team')

In [ ]:
train_data.isnull().sum()

Date                 0
Season               0
Match_ID             0
Game_ID              0
Home Team            0
Away Team            0
Score                0
Shots                0
SoT                  0
Accurate passes      0
Inaccurate passes    0
Passes               0
Start_minutes        0
End_minutes          0
Shot_Pct             0
Pass_Pct             0
Shots_ratio          0
SoT_ratio            0
acc_pass_ratio       0
inacc_pass_ratio     0
pass_ratio           0
Goals_scored         0
Goals_conceded       0
Goal_diff            0
Goals_ratio_home     0
dtype: int64

In [ ]:
train_data=pd.merge(train_data,goals2, how='inner', left_on='Away Team', right_on='Away Team')

In [ ]:
train_data.drop(['Date','Season','Match_ID','Home Team','Away Team','Game_ID'], axis=1, inplace=True)

In [ ]:
train_data['Goals_ratio']=train_data['Goals_ratio_home']/train_data['Goals_ratio_away']

TEST DATA

In [ ]:
test_game_statistics.drop(['next_player', 'next_x', 'next_y', 'event_id', 'next_team','next_event_id', 'xt_value'], axis=1, inplace=True)

In [ ]:
temp_data=list(zip(test.Game_ID,test['Home Team']))
temp_data

[('ID_8518U587', 'Antennae'),
 ('ID_H49BIKG7', 'Cosmos Redshift 7'),
 ('ID_PO6SP4VA', 'Cosmos Redshift 7'),
 ('ID_MZRCNBAQ', 'Cartwheel'),
 ('ID_CV9VOLIU', 'Cosmos Redshift 7'),
 ('ID_EP02V6O9', 'Cosmos Redshift 7'),
 ('ID_D1BKOMNB', 'Medusa Merger'),
 ('ID_GGFY5Z38', 'Andromeda'),
 ('ID_YQ6DCSJJ', 'Cigar'),
 ('ID_ZIMAODWU', 'Circinus'),
 ('ID_H6V0DHCM', 'Cosmos Redshift 7'),
 ('ID_EFUM1IB3', 'Cosmos Redshift 7'),
 ('ID_7H9RUYR2', 'Antennae'),
 ('ID_7JTHUMTF', 'Antennae'),
 ('ID_VZDE4BA3', 'Sunflower'),
 ('ID_1Y735BT9', 'Andromeda'),
 ('ID_C51DS9XE', 'Malin 1'),
 ('ID_EEAWBOYL', 'Fireworks'),
 ('ID_OKHY3GBQ', 'Cigar'),
 ('ID_JKJP61NL', 'Antennae'),
 ('ID_ZRZC49M1', 'Antennae'),
 ('ID_NQGKKQRG', 'Antennae'),
 ('ID_KL74YZ1W', 'Needle'),
 ('ID_D4BBONMS', 'Antennae'),
 ('ID_E4NAVOHG', 'Circinus'),
 ('ID_8AKT073X', 'Antennae'),
 ('ID_SGHWIP1E', 'Antennae'),
 ('ID_OHS6RR07', 'Milky Way'),
 ('ID_0BFR12C0', 'Antennae'),
 ('ID_O3EQ8WEA', 'Medusa Merger'),
 ('ID_PD70LTRE', 'Medusa Merger'),
 ('I

In [ ]:
home_data=[];
for x,y in temp_data:
    b=test_game_statistics[(test_game_statistics.Game_ID==x) & (test_game_statistics.Team==y)].values
    home_data.append(b)
home_data[0]

array([['ID_8518U587', 'Player_XY4WUS3P', 5.0, ..., 0.0, 0.12, 0.45],
       ['ID_8518U587', 'Player_XY4WUS3P', 9.0, ..., 0.0, 0.12, 0.45],
       ['ID_8518U587', 'Player_XY4WUS3P', 10.0, ..., 0.0, 0.12, 0.45],
       ...,
       ['ID_8518U587', 'Player_YCIF12TK', 3993.0, ..., 0.0, 97.93, 98.27],
       ['ID_8518U587', 'Player_5LKU609Q', 3995.0, ..., 0.0, 97.93, 98.27],
       ['ID_8518U587', 'Player_5LKU609Q', 3996.0, ..., 0.0, 97.93, 98.27]],
      dtype=object)

In [ ]:
# Away Team
temp_data=list(zip(test.Game_ID,test['Away Team']))
temp_data

[('ID_8518U587', 'Cosmos Redshift 7'),
 ('ID_H49BIKG7', 'Tadpole'),
 ('ID_PO6SP4VA', 'Medusa Merger'),
 ('ID_MZRCNBAQ', 'Cosmos Redshift 7'),
 ('ID_CV9VOLIU', 'Needle'),
 ('ID_EP02V6O9', 'Antennae'),
 ('ID_D1BKOMNB', 'Cosmos Redshift 7'),
 ('ID_GGFY5Z38', 'Cosmos Redshift 7'),
 ('ID_YQ6DCSJJ', 'Cosmos Redshift 7'),
 ('ID_ZIMAODWU', 'Cosmos Redshift 7'),
 ('ID_H6V0DHCM', 'Andromeda'),
 ('ID_EFUM1IB3', 'Sunflower'),
 ('ID_7H9RUYR2', 'Backward'),
 ('ID_7JTHUMTF', 'Cigar'),
 ('ID_VZDE4BA3', 'Antennae'),
 ('ID_1Y735BT9', 'Antennae'),
 ('ID_C51DS9XE', 'Antennae'),
 ('ID_EEAWBOYL', 'Antennae'),
 ('ID_OKHY3GBQ', 'Antennae'),
 ('ID_JKJP61NL', 'Needle'),
 ('ID_ZRZC49M1', 'Whirlpool'),
 ('ID_NQGKKQRG', 'Circinus'),
 ('ID_KL74YZ1W', 'Antennae'),
 ('ID_D4BBONMS', 'Medusa Merger'),
 ('ID_E4NAVOHG', 'Antennae'),
 ('ID_8AKT073X', 'Sunflower'),
 ('ID_SGHWIP1E', 'Fireworks'),
 ('ID_OHS6RR07', 'Antennae'),
 ('ID_0BFR12C0', 'Tadpole'),
 ('ID_O3EQ8WEA', 'Tadpole'),
 ('ID_PD70LTRE', 'Comet'),
 ('ID_B9JSESNH

In [ ]:
away_data=[];
for x,y in temp_data:
    b=test_game_statistics[(test_game_statistics.Game_ID==x) & (test_game_statistics.Team==y)].values
    away_data.append(b)
away_data[0]

array([['ID_8518U587', 'Player_K4YVUM05', 1.0, ..., 0.0, 0.03, 0.37],
       ['ID_8518U587', 'Player_K4YVUM05', 2.0, ..., 1.0, 0.03, 0.37],
       ['ID_8518U587', 'Player_XH7OO4D7', 3.0, ..., 0.0, 0.07, 0.4],
       ...,
       ['ID_8518U587', 'Player_930F2XZY', 3991.0, ..., 0.0, 97.93, 98.27],
       ['ID_8518U587', 'Player_930F2XZY', 3992.0, ..., 0.0, 97.93, 98.27],
       ['ID_8518U587', 'Player_930F2XZY', 3994.0, ..., 0.0, 97.93, 98.27]],
      dtype=object)

In [ ]:
# Convert Lists to DataFrames
# Home Data
b=[pd.DataFrame(home_data[x],columns=test_game_statistics.columns) for x in range(len(home_data))]
home_data1=pd.concat(b)
print(home_data1.shape)
# Away data
b=[pd.DataFrame(away_data[x],columns=test_game_statistics.columns) for x in range(len(away_data))]
away_data1=pd.concat(b)
print(away_data1.shape)

(393260, 17)
(386974, 17)


In [ ]:
game_home=home_data1.groupby(['Game_ID','Player_ID']).sum().groupby(['Game_ID']).sum()
game_home

,id,X,Y,Season,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes
Game_ID,,,,,,,,,,,
ID_03VRW8WV,3374113.0,94888.3,56636.9,5379,9.0,3.0,371.0,88.0,459.0,86846.45,87445.78
ID_05YW7A1R,3065867.0,84070.8,59595.1,5139,8.0,2.0,331.0,78.0,409.0,75976.57,76548.28
ID_064HV3UA,2987645.0,89237.7,54987.7,5274,10.0,4.0,352.0,72.0,424.0,78398.11,78984.92
ID_06SV1XSM,3887186.0,120749.5,71447.9,6444,8.0,2.0,513.0,77.0,590.0,106251.53,106968.05
ID_08N8KYTM,2426430.0,62844.7,40082.5,3993,9.0,2.0,213.0,57.0,270.0,60371.62,60816.51
...,...,...,...,...,...,...,...,...,...,...,...
ID_ZMO6O1V6,2679959.0,95435.4,66217.4,5073,11.0,5.0,336.0,93.0,429.0,77868.00,78432.53
ID_ZQOB9ZQL,3383625.0,102157.8,63276.6,5496,14.0,4.0,333.0,85.0,418.0,91984.09,92595.19
ID_ZRZC49M1,2813257.0,93169.2,53257.1,5256,9.0,3.0,402.0,81.0,483.0,80205.37,80789.13


In [ ]:
game_home.drop(['id','X','Y','Season'], axis=1, inplace=True)

In [ ]:
game_home['Shot_Pct']=((game_home['SoT']+1)/(game_home['Shots']+1))
game_home['Pass_Pct']=((game_home['Accurate passes']+1)/(game_home['Passes']+1))

In [ ]:
game_away=away_data1.groupby(['Game_ID','Player_ID']).sum().groupby(['Game_ID']).sum()
game_away

,id,X,Y,Season,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes
Game_ID,,,,,,,,,,,
ID_03VRW8WV,3341930.0,87805.4,68249.3,5610,5.0,3.0,385.0,88.0,473.0,85621.35,86246.14
ID_05YW7A1R,3775384.0,109572.1,69656.6,5952,6.0,1.0,412.0,93.0,505.0,93700.59,94362.22
ID_064HV3UA,3225101.0,92542.5,63448.8,5295,11.0,4.0,365.0,59.0,424.0,84823.32,85412.11
ID_06SV1XSM,2481613.0,72230.9,48140.1,4257,4.0,0.0,256.0,76.0,332.0,67796.81,68270.97
ID_08N8KYTM,4992429.0,153640.8,86799.0,7557,19.0,7.0,611.0,80.0,691.0,124845.82,125685.81
...,...,...,...,...,...,...,...,...,...,...,...
ID_ZMO6O1V6,2712360.0,77413.9,53983.3,4773,4.0,0.0,338.0,80.0,418.0,78746.03,79278.11
ID_ZQOB9ZQL,2839748.0,85631.2,53784.7,5082,9.0,3.0,282.0,74.0,356.0,77169.90,77735.71
ID_ZRZC49M1,2438544.0,75803.5,50774.6,4461,8.0,0.0,313.0,64.0,377.0,69984.17,70479.33


In [ ]:
game_away.drop(['id','X','Y','Season'], axis=1, inplace=True)

In [ ]:
game_away['Shot_Pct']=((game_away['SoT']+1)/(game_away['Shots']+1))
game_away['Pass_Pct']=((game_away['Accurate passes']+1)/(game_away['Passes']+1))

In [ ]:
game_away.head()

,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,Shot_Pct,Pass_Pct
Game_ID,,,,,,,,,
ID_03VRW8WV,5.0,3.0,385.0,88.0,473.0,85621.35,86246.14,0.666667,0.814346
ID_05YW7A1R,6.0,1.0,412.0,93.0,505.0,93700.59,94362.22,0.285714,0.816206
ID_064HV3UA,11.0,4.0,365.0,59.0,424.0,84823.32,85412.11,0.416667,0.861176
ID_06SV1XSM,4.0,0.0,256.0,76.0,332.0,67796.81,68270.97,0.200000,0.771772
ID_08N8KYTM,19.0,7.0,611.0,80.0,691.0,124845.82,125685.81,0.400000,0.884393


In [ ]:
game_test_combined=game_home

In [ ]:
game_test_combined['Shots_ratio']=(game_home['Shots']+1)/(game_away['Shots']+1)
game_test_combined['SoT_ratio']=(game_home['SoT']+1)/(game_away['SoT']+1)
game_test_combined['acc_pass_ratio']=(game_home['Accurate passes']+1)/(game_away['Accurate passes']+1)
game_test_combined['inacc_pass_ratio']=(game_home['Inaccurate passes']+1)/(game_away['Inaccurate passes']+1)
game_test_combined['pass_ratio']=(game_home['Passes']+1)/(game_away['Passes']+1)
game_test_combined['Shotpct_ratio']=(game_home['Shot_Pct'])/(game_away['Shot_Pct'])
game_test_combined['passpct_ratio']=(game_home['Pass_Pct'])/(game_away['Pass_Pct'])

In [ ]:
game_test_combined.head()

,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,Shot_Pct,Pass_Pct,Shots_ratio,SoT_ratio,acc_pass_ratio,inacc_pass_ratio,pass_ratio,Shotpct_ratio,passpct_ratio
Game_ID,,,,,,,,,,,,,,,,
ID_03VRW8WV,9.0,3.0,371.0,88.0,459.0,86846.45,87445.78,0.400000,0.808696,1.666667,1.000,0.963731,1.000000,0.970464,0.600000,0.993062
ID_05YW7A1R,8.0,2.0,331.0,78.0,409.0,75976.57,76548.28,0.333333,0.809756,1.285714,1.500,0.803874,0.840426,0.810277,1.166667,0.992098
ID_064HV3UA,10.0,4.0,352.0,72.0,424.0,78398.11,78984.92,0.454545,0.830588,0.916667,1.000,0.964481,1.216667,1.000000,1.090909,0.964481
ID_06SV1XSM,8.0,2.0,513.0,77.0,590.0,106251.53,106968.05,0.333333,0.869712,1.800000,3.000,2.000000,1.012987,1.774775,1.666667,1.126904
ID_08N8KYTM,9.0,2.0,213.0,57.0,270.0,60371.62,60816.51,0.300000,0.789668,0.500000,0.375,0.349673,0.716049,0.391618,0.750000,0.892892


In [ ]:
goals1

,Home Team,Goals_scored,Goals_conceded,Goal_diff,Goals_ratio_home
0,Andromeda,47.0,48.0,-1.0,0.979167
1,Antennae,53.0,50.0,3.0,1.060000
2,Backward,41.0,51.0,-10.0,0.803922
3,Butterfly,28.0,21.0,7.0,1.333333
4,Cartwheel,72.0,72.0,0.0,1.000000
5,Cigar,78.0,58.0,20.0,1.344828
6,Circinus,32.0,56.0,-24.0,0.571429
7,Coma Pinwheel,42.0,73.0,-31.0,0.575342
8,Comet,55.0,59.0,-4.0,0.932203
9,Cosmos Redshift 7,66.0,46.0,20.0,1.434783


In [ ]:
goals1.drop([14,3,10,7,15,18], axis=0, inplace=True)
goals2.drop([14,3,10,7,15,18], axis=0, inplace=True)

In [ ]:
goals1.head()

,Home Team,Goals_scored,Goals_conceded,Goal_diff,Goals_ratio_home
0,Andromeda,47.0,48.0,-1.0,0.979167
1,Antennae,53.0,50.0,3.0,1.060000
2,Backward,41.0,51.0,-10.0,0.803922
4,Cartwheel,72.0,72.0,0.0,1.000000
5,Cigar,78.0,58.0,20.0,1.344828


In [ ]:
goals2.head()

,Away Team,Goals_ratio_away
0,Andromeda,0.979167
1,Antennae,1.060000
2,Backward,0.803922
4,Cartwheel,1.000000
5,Cigar,1.344828


In [ ]:
data={'Home Team':['Malin 1','Needle','Whirlpool'], 'Goals_ratio_home':[1,1,1]}

In [ ]:
df=pd.DataFrame(data)

In [ ]:
goals1=pd.concat([goals1,df],axis=0)

In [ ]:
goals1

,Home Team,Goals_scored,Goals_conceded,Goal_diff,Goals_ratio_home
0,Andromeda,47.0,48.0,-1.0,0.979167
1,Antennae,53.0,50.0,3.0,1.060000
2,Backward,41.0,51.0,-10.0,0.803922
4,Cartwheel,72.0,72.0,0.0,1.000000
5,Cigar,78.0,58.0,20.0,1.344828
6,Circinus,32.0,56.0,-24.0,0.571429
8,Comet,55.0,59.0,-4.0,0.932203
9,Cosmos Redshift 7,66.0,46.0,20.0,1.434783
11,Fireworks,48.0,64.0,-16.0,0.750000
12,Medusa Merger,90.0,35.0,55.0,2.571429


In [ ]:
data1={'Away Team':['Malin 1','Needle','Whirlpool'], 'Goals_ratio_away':[1,1,1]}

In [ ]:
df1=pd.DataFrame(data1)

In [ ]:
goals2=pd.concat([goals2,df1],axis=0)

In [ ]:
goals1.shape, goals2.shape

((16, 5), (16, 2))

In [ ]:
goals1.head()

,Home Team,Goals_scored,Goals_conceded,Goal_diff,Goals_ratio_home
0,Andromeda,47.0,48.0,-1.0,0.979167
1,Antennae,53.0,50.0,3.0,1.060000
2,Backward,41.0,51.0,-10.0,0.803922
4,Cartwheel,72.0,72.0,0.0,1.000000
5,Cigar,78.0,58.0,20.0,1.344828


In [ ]:
goals1.drop(['Goals_scored','Goals_conceded','Goal_diff'], axis=1, inplace=True)

In [ ]:
goals2

,Away Team,Goals_ratio_away
0,Andromeda,0.979167
1,Antennae,1.060000
2,Backward,0.803922
4,Cartwheel,1.000000
5,Cigar,1.344828
6,Circinus,0.571429
8,Comet,0.932203
9,Cosmos Redshift 7,1.434783
11,Fireworks,0.750000
12,Medusa Merger,2.571429


In [ ]:
goals1=goals1.reset_index()
goals2=goals2.reset_index()

In [ ]:
goals1.head()

,index,Home Team,Goals_ratio_home
0,0,Andromeda,0.979167
1,1,Antennae,1.060000
2,2,Backward,0.803922
3,4,Cartwheel,1.000000
4,5,Cigar,1.344828


In [ ]:
goals1.drop(['index'], axis=1, inplace=True)
goals2.drop(['index'], axis=1, inplace=True)

,Away Team,Goals_ratio_away
0,Andromeda,0.979167
1,Antennae,1.060000
2,Backward,0.803922
4,Cartwheel,1.000000
5,Cigar,1.344828
6,Circinus,0.571429
8,Comet,0.932203
9,Cosmos Redshift 7,1.434783
11,Fireworks,0.750000
12,Medusa Merger,2.571429


In [ ]:
test_data=pd.merge(test,game_test_combined,left_on='Game_ID',right_index=True,how='inner')

In [ ]:
test_data.head()

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Shots,SoT,Accurate passes,Inaccurate passes,...,End_minutes,Shot_Pct,Pass_Pct,Shots_ratio,SoT_ratio,acc_pass_ratio,inacc_pass_ratio,pass_ratio,Shotpct_ratio,passpct_ratio
0,2018-12-08,3,102.0,ID_8518U587,Antennae,Cosmos Redshift 7,11.0,1.0,312.0,92.0,...,89380.91,0.166667,0.772840,0.750000,0.333333,0.772840,1.148148,0.835052,0.444444,0.925499
1,2018-09-11,3,18.0,ID_H49BIKG7,Cosmos Redshift 7,Tadpole,6.0,2.0,335.0,72.0,...,83897.43,0.428571,0.823529,0.777778,1.000000,1.191489,0.858824,1.114754,1.285714,1.068836
2,2018-09-25,3,32.0,ID_PO6SP4VA,Cosmos Redshift 7,Medusa Merger,7.0,3.0,259.0,73.0,...,75349.18,0.500000,0.780781,0.666667,1.333333,0.626506,1.193548,0.699580,2.000000,0.895546
3,2018-10-02,3,40.0,ID_MZRCNBAQ,Cartwheel,Cosmos Redshift 7,9.0,0.0,447.0,76.0,...,102206.65,0.100000,0.854962,1.111111,0.200000,1.302326,0.974684,1.241706,0.180000,1.048819
4,2018-12-20,3,123.0,ID_CV9VOLIU,Cosmos Redshift 7,Needle,6.0,1.0,327.0,77.0,...,80725.08,0.285714,0.809877,0.700000,0.666667,0.657315,0.876404,0.689949,0.952381,0.952700


In [ ]:
test_data=pd.merge(test_data,goals1, how='inner', left_on='Home Team', right_on='Home Team')

In [ ]:
test_data=pd.merge(test_data,goals2, how='inner', left_on='Away Team', right_on='Away Team')

In [ ]:
test_data.isnull().sum()

Date                 0
Season               0
Match_ID             0
Game_ID              0
Home Team            0
Away Team            0
Shots                0
SoT                  0
Accurate passes      0
Inaccurate passes    0
Passes               0
Start_minutes        0
End_minutes          0
Shot_Pct             0
Pass_Pct             0
Shots_ratio          0
SoT_ratio            0
acc_pass_ratio       0
inacc_pass_ratio     0
pass_ratio           0
Shotpct_ratio        0
passpct_ratio        0
Goals_ratio_home     0
Goals_ratio_away     0
dtype: int64

In [ ]:
test_data.shape

(234, 24)

In [ ]:
train_data.columns

Index(['Score', 'Shots', 'SoT', 'Accurate passes', 'Inaccurate passes',
       'Passes', 'Start_minutes', 'End_minutes', 'Shot_Pct', 'Pass_Pct',
       'Shots_ratio', 'SoT_ratio', 'acc_pass_ratio', 'inacc_pass_ratio',
       'pass_ratio', 'Goals_ratio_home', 'Goals_ratio_away', 'Goals_ratio'],
      dtype='object')

In [ ]:
test_data.columns

Index(['Date', 'Season', 'Match_ID', 'Game_ID', 'Home Team', 'Away Team',
       'Shots', 'SoT', 'Accurate passes', 'Inaccurate passes', 'Passes',
       'Start_minutes', 'End_minutes', 'Shot_Pct', 'Pass_Pct', 'Shots_ratio',
       'SoT_ratio', 'acc_pass_ratio', 'inacc_pass_ratio', 'pass_ratio',
       'Shotpct_ratio', 'passpct_ratio', 'Goals_ratio_home',
       'Goals_ratio_away'],
      dtype='object')

In [ ]:
test_data['Goals_ratio']=test_data['Goals_ratio_home']/test_data['Goals_ratio_away']

In [ ]:
test_data.shape

(234, 25)

In [ ]:
test_data.drop(['Date','Season','Match_ID','Home Team','Away Team'], axis=1, inplace=True)

In [ ]:
train_data.shape, test_data.shape

((412, 21), (234, 20))

In [ ]:
train_data.columns

Index(['Score', 'Shots', 'SoT', 'Accurate passes', 'Inaccurate passes',
       'Passes', 'Start_minutes', 'End_minutes', 'Shot_Pct', 'Pass_Pct',
       'Shots_ratio', 'SoT_ratio', 'acc_pass_ratio', 'inacc_pass_ratio',
       'pass_ratio', 'Goals_scored', 'Goals_conceded', 'Goal_diff',
       'Goals_ratio_home', 'Goals_ratio_away', 'Goals_ratio'],
      dtype='object')

In [ ]:
test_data.columns

Index(['Game_ID', 'Shots', 'SoT', 'Accurate passes', 'Inaccurate passes',
       'Passes', 'Start_minutes', 'End_minutes', 'Shot_Pct', 'Pass_Pct',
       'Shots_ratio', 'SoT_ratio', 'acc_pass_ratio', 'inacc_pass_ratio',
       'pass_ratio', 'Shotpct_ratio', 'passpct_ratio', 'Goals_ratio_home',
       'Goals_ratio_away', 'Goals_ratio'],
      dtype='object')

In [ ]:
test_data.drop(['Game_ID','Shotpct_ratio','passpct_ratio'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_train=label_encoder.fit_transform(train_data.Score)
y_train


array([0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 2, 1, 1, 2, 1, 2, 1, 0, 0, 2, 2, 1, 1, 2, 0, 1, 0, 0, 0, 2, 2,
       0, 2, 2, 1, 0, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 2, 1, 0,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2,
       1, 1, 2, 2, 2, 0, 1, 2, 2, 0, 0, 1, 1, 0, 0, 2, 1, 0, 2, 2, 1, 2,
       0, 2, 0, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 1, 2, 0, 2, 2, 1, 1, 1, 2, 1, 2, 0, 2, 2, 2, 1, 0, 1,
       0, 0, 2, 1, 1, 2, 0, 2, 0, 1, 2, 1, 1, 0, 2, 0, 0, 0, 0, 1, 1, 0,
       2, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 2, 2, 0, 2, 1,
       2, 1, 0, 0, 2, 2, 2, 2, 0, 1, 1, 2, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0,
       2, 1, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 1, 2, 1,
       2, 2, 1, 1, 2, 2, 2, 0, 1, 1, 1, 0, 2, 0, 0, 0, 1, 2, 0, 0, 2, 1,
       1, 1, 2, 0, 0, 0, 0, 2, 0, 2, 1, 0, 1, 0, 0, 2, 1, 0, 2, 1, 1, 2,
       2, 1, 2, 0, 1, 2, 2, 0, 0, 2, 1, 1, 2, 1, 2,

In [ ]:
label_encoder.classes_

array(['Away win', 'Draw', 'Home Win'], dtype=object)

In [ ]:
# One Hot Encode Labels
from sklearn.preprocessing import OneHotEncoder
one_hot=OneHotEncoder()
y=one_hot.fit_transform(y_train.reshape(-1,1)).toarray()
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [ ]:
train_data.drop(['Score'], axis=1, inplace=True)


In [ ]:
train_data.shape, test_data.shape

((412, 20), (234, 17))

In [ ]:
train_data.drop(['Goals_scored','Goals_conceded','Goal_diff'], axis=1, inplace=True)

In [ ]:
test_data.columns

Index(['Shots', 'SoT', 'Accurate passes', 'Inaccurate passes', 'Passes',
       'Start_minutes', 'End_minutes', 'Shot_Pct', 'Pass_Pct', 'Shots_ratio',
       'SoT_ratio', 'acc_pass_ratio', 'inacc_pass_ratio', 'pass_ratio',
       'Goals_ratio_home', 'Goals_ratio_away', 'Goals_ratio'],
      dtype='object')

In [ ]:
train_data.shape, test_data.shape

((412, 17), (234, 17))

In [ ]:
from sklearn.preprocessing import StandardScaler,PowerTransformer,MinMaxScaler,OneHotEncoder,LabelEncoder
min_max=MinMaxScaler()
min_max.fit(train_data)
train=min_max.transform(train_data)
test=min_max.transform(test_data)
train.shape

(412, 17)

In [ ]:
from keras import layers
from tensorflow.keras import models
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_dim=17))
#model.add(layers.Dense(64, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dense(16, activation='relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

In [ ]:

model.compile(optimizer='adam',
loss='categorical_crossentropy',
metrics='categorical_crossentropy')

In [ ]:
train

array([[0.25      , 0.2       , 0.20141343, ..., 0.25754386, 0.21783626,
        0.19039271],
       [0.        , 0.        , 0.39575972, ..., 0.25754386, 0.21783626,
        0.19039271],
       [0.375     , 0.3       , 0.44169611, ..., 0.39181287, 0.21783626,
        0.2511874 ],
       ...,
       [0.25      , 0.3       , 0.24734982, ..., 0.13175095, 0.14619883,
        0.16472723],
       [0.0625    , 0.        , 0.38869258, ..., 0.01754386, 0.14619883,
        0.10396681],
       [0.34375   , 0.3       , 0.34982332, ..., 0.01946647, 0.14619883,
        0.10498968]])

In [ ]:
history = model.fit(train,
y,
epochs=50,
batch_size=52,
verbose=2,
validation_split=0.2)

Epoch 1/50
7/7 - 3s - loss: 1.1687 - categorical_crossentropy: 1.1687 - val_loss: 1.1184 - val_categorical_crossentropy: 1.1184 - 3s/epoch - 375ms/step
Epoch 2/50
7/7 - 0s - loss: 1.1087 - categorical_crossentropy: 1.1087 - val_loss: 1.1054 - val_categorical_crossentropy: 1.1054 - 39ms/epoch - 6ms/step
Epoch 3/50
7/7 - 0s - loss: 1.0867 - categorical_crossentropy: 1.0867 - val_loss: 1.1037 - val_categorical_crossentropy: 1.1037 - 34ms/epoch - 5ms/step
Epoch 4/50
7/7 - 0s - loss: 1.0742 - categorical_crossentropy: 1.0742 - val_loss: 1.1053 - val_categorical_crossentropy: 1.1053 - 42ms/epoch - 6ms/step
Epoch 5/50
7/7 - 0s - loss: 1.0641 - categorical_crossentropy: 1.0641 - val_loss: 1.0946 - val_categorical_crossentropy: 1.0946 - 36ms/epoch - 5ms/step
Epoch 6/50
7/7 - 0s - loss: 1.0535 - categorical_crossentropy: 1.0535 - val_loss: 1.0821 - val_categorical_crossentropy: 1.0821 - 35ms/epoch - 5ms/step
Epoch 7/50
7/7 - 0s - loss: 1.0415 - categorical_crossentropy: 1.0415 - val_loss: 1.0791

In [ ]:
test1=pd.read_csv('/content/gdrive/MyDrive/Laduma/Test_Laduma.csv',na_values='NULL')

In [ ]:

sub_data['Game_ID']=test1.Game_ID
sub_data[sub_data.columns[1:]]=model.predict(test)
sub_data

,Game_ID,Away win,Draw,Home Win
0,ID_8518U587,0.282581,0.431157,0.286262
1,ID_H49BIKG7,0.378502,0.353600,0.267898
2,ID_PO6SP4VA,0.077838,0.397704,0.524458
3,ID_MZRCNBAQ,0.199930,0.388874,0.411196
4,ID_CV9VOLIU,0.084595,0.322606,0.592799
...,...,...,...,...
229,ID_QKQ00O16,0.583843,0.245853,0.170304
230,ID_172V2IXW,0.053579,0.293121,0.653299
231,ID_2ZK5TE39,0.312580,0.441964,0.245456
232,ID_IS54QGW9,0.219447,0.383617,0.396936


In [ ]:
sub_data.to_csv('/content/gdrive/MyDrive/Laduma/DL.csv',index=False)
